## Análisis Exploratorio de Datos - Propensión de Compra

In [2]:
# librerías
import pandas as pd

In [ ]:
# Ruta de exportación
csv_path = r"C:\Users\Claudia\OneDrive - Universidad Alfonso X el Sabio\Documentos\GitHub\Caso01_Propensity\data.csv"

# Cargar los datos en un DataFrame
df = pd.read_csv(csv_path, delimiter=',')

In [4]:
# Número de filas y columnas
print(df.shape)  

(58049, 24)


In [6]:
# Primeras filas del DataFrame
df.head()

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,...,Medio,NO DATA,No,NO,1376,0,True,4,21,3
3,CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,...,Medio,SI,Averia muy grave,SI,2015,7256,True,4,48,5
4,CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,...,Alto,NO DATA,No,NO,1818,0,True,3,21,3


In [8]:

df.dtypes

CODE               object
PRODUCTO           object
TIPO_CARROCERIA    object
COMBUSTIBLE        object
Potencia           object
TRANS              object
FORMA_PAGO         object
ESTADO_CIVIL       object
GENERO             object
OcupaciOn          object
PROVINCIA          object
Campanna1          object
Campanna2          object
Campanna3          object
Zona_Renta         object
REV_Garantia       object
Averia_grave       object
QUEJA_CAC          object
COSTE_VENTA         int64
km_anno             int64
Mas_1_coche          bool
Revisiones          int64
Edad_Cliente        int64
Tiempo              int64
dtype: object

In [9]:
df.columns

Index(['CODE', 'PRODUCTO', 'TIPO_CARROCERIA', 'COMBUSTIBLE', 'Potencia',
       'TRANS', 'FORMA_PAGO', 'ESTADO_CIVIL', 'GENERO', 'OcupaciOn',
       'PROVINCIA', 'Campanna1', 'Campanna2', 'Campanna3', 'Zona_Renta',
       'REV_Garantia', 'Averia_grave', 'QUEJA_CAC', 'COSTE_VENTA', 'km_anno',
       'Mas_1_coche', 'Revisiones', 'Edad_Cliente', 'Tiempo'],
      dtype='object')

### Descripción de las Variables del Dataset

| **Variable**      | **Descripción** |
|------------------|------------------------------------------------|
| `CODE`          | Código único del cliente. |
| `PRODUCTO`      | Modelo de coche adquirido. |
| `TIPO_CARROCERIA` | Tipo de carrocería del coche (SUV, Sedán, etc.). |
| `COMBUSTIBLE`   | Tipo de combustible utilizado (Gasolina, Diésel, etc.). |
| `Potencia`      | Potencia del coche en caballos de fuerza. |
| `TRANS`         | Tipo de transmisión (Manual o Automática). |
| `FORMA_PAGO`    | Método de pago (Financiado o Contado). |
| `ESTADO_CIVIL`  | Estado civil del cliente al momento de la compra. |
| `GENERO`        | Género del cliente (Masculino o Femenino). |
| `OcupaciOn`     | Ocupación del cliente en el momento de la compra. |
| `PROVINCIA`     | Provincia de residencia del cliente. |
| `Campanna1`     | Si el cliente recibió un descuento por financiar (Sí/No). |
| `Campanna2`     | Si el cliente recibió un descuento por adquirir un seguro (Sí/No). |
| `Campanna3`     | Si el cliente recibió un descuento por extender la garantía (Sí/No). |
| `Zona_Renta`    | Nivel de renta promedio en la zona de residencia del cliente. |
| `REV_Garantia`  | Si el coche sigue en garantía. |
| `Averia_grave`  | Si el coche ha tenido una avería grave y de qué tipo. |
| `QUEJA_CAC`     | Si el cliente ha contactado con atención al cliente por alguna queja. |
| `COSTE_VENTA`   | Suma de descuentos aplicados en la venta. |
| `km_anno`       | Kilometraje medio recorrido por el coche al año. |
| `Mas_1_coche`   | Indica si el cliente ha comprado más de un coche. |
| `Revisiones`    | Número de revisiones pasadas del coche. |
| `Edad_Cliente`  | Edad actual del cliente. |
| `Tiempo`        | Años transcurridos entre compras de coches por el cliente. |



In [10]:
df.describe(include='all')

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
count,58049,58049,58049,58049,58049,58049,58049,57159,57189,58049,...,44871,58049,58048,58049,58049.000000,58049.000000,58049,58049.000000,58049.000000,58049.000000
unique,58049,11,8,2,3,2,4,4,2,3,...,4,2,4,2,NaN,NaN,2,NaN,NaN,NaN
top,CLI58048,B,TIPO1,FUEL 2,Media,M,Contado,CASADO,M,Empresa,...,Alto,NO DATA,No,NO,NaN,NaN,False,NaN,NaN,NaN
freq,1,15993,23387,33085,39906,52969,30190,43529,40170,53262,...,17152,32011,29935,36268,NaN,NaN,41107,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2540.125756,11805.893607,NaN,3.529587,47.351255,1.859119
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1603.556511,10205.363971,NaN,2.528413,11.239984,3.091309
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,18.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1595.000000,0.000000,NaN,1.000000,40.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2353.000000,11486.000000,NaN,3.000000,48.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3301.000000,17927.000000,NaN,5.000000,56.000000,4.000000


### Limpieza de datos.

In [7]:
# Valores nulos por columna
print(df.isnull().sum()) 

CODE                   0
PRODUCTO               0
TIPO_CARROCERIA        0
COMBUSTIBLE            0
Potencia               0
TRANS                  0
FORMA_PAGO             0
ESTADO_CIVIL         890
GENERO               860
OcupaciOn              0
PROVINCIA              0
Campanna1              0
Campanna2              0
Campanna3              0
Zona_Renta         13178
REV_Garantia           0
Averia_grave           1
QUEJA_CAC              0
COSTE_VENTA            0
km_anno                0
Mas_1_coche            0
Revisiones             0
Edad_Cliente           0
Tiempo                 0
dtype: int64
